In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.7:4041
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1642841918971)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1046b641


Dataset은 구조적 API의 기본 데이터 타입임<br/>
앞에서 이미 DataFrame을 다루었음<br/>
DataFrame은 Row 타입의 Dataset이며 스파크가 지원하는 다양한 언어에서 사용할 수 있음<br/>
(스파크 소스 코드 중 org/apache/spark/sql/package.scala 파일을 보면 type DataFrame = Dataset\[Row\]로 선언되어 있음)<br/>
**Dataset은 자바 가상 머신을 사용하는 언어인 스칼라와 자바에서만 사용 가능함**<br/>
Dataset을 사용해 데이터셋의 각 로우를 구성하는 객체를 정의함<br/>
스칼라에서는 스키마가 정의된 케이스 클래스 객체를 사용해 Dataset을 정의함<br/>
자바에서는 자바 빈 객체를 사용해 Dataset을 정의함<br/>

스파크는 StringType, BigIntType, StructType과 같은 다양한 데이터 타입을 제공함<br/>
또한 스파크가 지원하는 다양한 언어의 String, Integer 그리고 Double과 같은 데이터 타입을 스파크의 특정 데이터 타입으로 매핑할 수 있음<br/>
DataFrame API를 사용할 때 String이나 Integer 데이터 타입의 객체를 생성하지는 않지만 스파크는 Row 객체를 변환해 데이터를 처리함<br/>
**사실 스칼라나 자바를 사용할 때 모든 DataFrame은 Row 타입의 Dataset을 의미함**<br/>
도메인별 특정 객체를 효과적으로 지원하기 위해 encoder라고 부르는 특수 개념이 필요함<br/>
encoder는 도메인별 특정 객체 T를 스파크의 내부 데이터 타입으로 매핑하는 시스템을 의미함<br/>


예를 들어 name(string)과 age(int) 두 개의 필드를 가진 Person 클래스가 있다고 가정하자<br/>
encoder는 런타임 환경에서 Person 객체를 바이너리 구조로 직렬화하는 코드를 생성하도록 스파크에 지시함<br/>
DataFrame이나 '표준' 구조적 API를 사용한다면 Row 타입을 직렬화된 바이너리 구조로 변환함<br/>
도메인에 특화된 객체를 만들어 사용하려면 스칼라의 케이스 클래스(case class) 또는 자바의 자바빈(JavaBean) 형태로 사용자 정의 데이터 타입을 정의해야 함<br/>
스파크에서는 Row 타입 대신 사용자가 정의한 데이터 타입을 분산 방식으로 다룰 수 있음<br/>

**Dataset API를 사용한다면 스파크는 데이터셋에 접근할 때마다 Row 포맷이 아닌 사용자 정의 데이터 타입으로 변환함<br/>
이 변환 작업은 느리긴 하지만 사용자에게 더 많은 유연성을 제공할 수 있음<br/>
사용자 정의 데이터 타입을 사용하면 성능이 나빠지게 됨<br/>
파이썬으로 만든 사용자 정의 함수와 비슷한 상황으로 볼 수 있으나 자릿수가 다를 정도로 사용자 정의 함수가 훨씬 더 느림<br/>
그 이유는 프로그래밍 언어를 전환하는 것이 사용자 정의 데이터 타입을 사용하는 것보다 훨씬 더 느리기 때문**<br/>

# 11.1 Dataset을 사용할 시기

Dataset을 사용하면 성능이 떨어진다는데 사용할 필요가 있을까?<br/>
Dataset을 사용해야 하는 2가지 이유<br/>
* DataFrame 기능만으로 수행할 연산을 표현할 수 없는 경우
* 성능 저하를 감수하더라도 타입 안정성(type-safety)을 가진 데이터 타입을 사용하고 싶은 경우

Dataset을 사용해야 하는 이유를 자세히 알아보겠음<br/>
구조적 API를 사용해 표현할 수 없는 몇 가지 작업이 있음<br/>
흔하진 않지만, 복잡한 비즈니스 로직을 SQL이나 DataFrame 대신 단일 함수로 인코딩해야 하는 경우가 있음<br/>
이런 경우 Dataset을 사용하는 것이 적합함<br/>
또한 Dataset API는 타입 안정성이 있음<br/>
예를 들어 두 문자열을 사용해 뺄셈 연산을 하는 것처럼 데이터 타입이 유효하지 않은 작업은 런타임이 아닌 컴파일 타임에 요류가 발생함<br/>

단일 노드의 워크로드와 스파크 워크로드에서 전체 로우에 대한 다양한 transformation을 재사용하려면 Dataset을 사용하는 것이 적합함<br/>
*스파크의 API는 스칼라 Sequence 타입의 API가 일부 반영되어 있지만 분산 방식으로 동작함*<br/>
결국 Dataset을 사용하는 장점 중 하나는 로컬과 분산 환경의 워크로드에서 재사용할 수 있다는 것임<br/>
케이스 클래스로 구현된 데이터 타입을 사용해 모든 데이터와 transformation을 정의하면 재사용할 수 있음<br/>
또한 올바른 클래스와 데이터 타입이 지정된 DataFrame을 로컬 디스크에 저장하면 다음 처리 과정에서 사용할 수 있어 더 쉽게 데이터를 다룰 수 있음<br/>

더 적합한 워크로드를 만들기 위해 DataFrame과 Dataset을 동시에 사용해야 할 때가 있음<br/>
하지만 성능과 타입 안정성 중 하나는 반드시 희생할 수밖에 없음<br/>
*이러한 방식은 대량의 DataFrame 기반의 ETL transformation의 마지막 단계에서 사용할 수 있음<br/>
예를 들어 드라이버로 데이터를 수집해 단일 노드의 라이브러리로 수집된 데이터를 처리하는 경우*<br/>
반대로 transformation의 첫 번째 단계에서 사용할 수도 있음 (예를 들어 스파크 SQL에서 필터링 전에 로우 단위로 데이터를 파싱하는 경우)<br/>

# 11.2 Dataset 생성

Dataset을 생성하는 것은 수동 작업이므로 정의할 스키마를 미리 알고 있어야 함<br/>

## 11.2.1 자바: Encoders

자바 인코더는 매우 간단함<br/>
데이터 타입 클래스를 정의한 다음 DataFrame(Dataset\< Row\> 타입의)에 지정해 인코딩할 수 있음<br/>

In [6]:
/*
// 자바 코드

import org.apache.spark.sql.Encoders;

public class Flight implements Serializable{
    String DEST_COUNTRY_NAME;
    String ORIGIN_COUNTRY_NAME;
    Long DEST_COUNTRY_NAME;
}

Dataset<Flight> flights = spark.read
    .parquet("Downloads/Spark-The-Definitive-Guide/data/flight-data/parquet/2010-summary.parquet/")
    .as(Encoders.bean(Flight.class));
*/

## 11.2.2 스칼라: 케이스 클래스

스칼라에서 Dataset을 생성하려면 스칼라 case class 구문을 사용해 데이터 타입을 정의해야 함<br/>
케이스 클래스는 다음과 같은 특징을 가진 정규 클래스(regular class)임<br/>
* 불변성
* 패턴 매칭으로 분해 가능
* 참조값 대신 클래스 구조를 기반으로 비교
* 사용하기 쉽고 다루기 편함

이러한 특징으로 케이스 클래스를 판별할 수 있으므로 데이터 분석 시 유용함<br/>

스칼라 문서는 케이스 클래스를 다음과 같이 설명함<br/>
* 불변성이므로 객체들이 언제 어디서 변경되었는지 추적할 필요가 없음<br/>
* 패턴 매칭은 로직 분기를 단순화해 버그를 줄이고 가독성을 좋게 만듦<br/>

Dataset을 생성하기 위해 예제 데이터셋 중 하나를 case class로 정의함<br/>

In [7]:
case class Flight(DEST_COUNTRY_NAME: String, ORIGIN_COUNTRY_NAME: String, count: BigInt)

defined class Flight


앞서 데이터셋의 레코드를 표현할 case class를 정의했음<br/>
즉, Flight 데이터 타입의 Dataset을 생성했음<br/>
Flight 데이터 타입은 스키마만 정의되어 있을 뿐 아무런 메서드도 정의되어 있지 않음<br/>
데이터를 읽으면 DataFrame이 반환됨<br/>
그리고 as 메서드를 사용해 Flights 데이터 타입으로 변환함<br/>

In [8]:
val flightsDF = spark.read
    .parquet("Downloads/Spark-The-Definitive-Guide/data/flight-data/parquet/2010-summary.parquet")
val flights = flightsDF.as[Flight]

flightsDF: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]
flights: org.apache.spark.sql.Dataset[Flight] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


# 11.3 액션

Dataset과 DataFrame에 collect, take 그리고 count 같은 액션을 적용할 수 있음<br/>

In [9]:
flights.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
+-----------------+-------------------+-----+
only showing top 2 rows



또한 케이스 클래스에 실제로 접근할 때 어떠한 데이터 타입도 필요하지 않다는 사실을 알고 있어야 함<br/>
case class의 속성명을 지정하면 속성에 맞는 값과 데이터 타입 모두를 반환함<br/>

In [10]:
flights.first.DEST_COUNTRY_NAME // United States

res3: String = United States


# 11.4 트랜스포메이션

Dataset의 트랜스포메이션은 DataFrame과 동일함<br/>
DataFrame의 모든 트랜스포메이션은 Dataset에서 사용할 수 있음<br/>

**Dataset을 사용하면 원형의 JVM 데이터 타입을 다루기 때문에 DataFrame만 사용해서 트랜스포메이션을 수행하는 것보다 좀 더 복잡하고 강력한 데이터 타입으로 트랜스포메이션을 사용할 수 있음**<br/>
원형 객체를 다루는 방법을 설명하기 위해 이전 예제에서 만든 Dataset에 필터를 적용해보겠음<br/>

## 11.4.1 필터링

Flight 클래스를 파라미터로 사용해 출발지와 도착지가 동일한지를 나타내는 불리언값을 반환하는 함수를 만들어보겠음<br/>
이 함수는 사용자 정의 함수(스파크 SQL은 예제와 같은 방식으로 사용자 정의 함수를 정의함)가 아닌 일반 함수임<br/>

*TIP*<br/>
다음 예제에서는 필터가 정의된 **함수**를 생성함<br/>
이 방식은 지금까지 했던 방식과는 다르므로 매우 중요함<br/>
스파크는 정의된 함수를 사용해 모든 로우를 평가하고, 따라서 매우 많은 자원을 사용함<br/>
그러므로 단순 필터라면 SQL 표현식을 사용하는 것이 좋음<br/>
SQL 표현식을 사용하면 데이터 필터링 비용이 크게 줄어들 뿐만 아니라 다음 처리 과정에서 Dataset으로 데이터를 다룰 수 있음<br/>

In [11]:
def originIsDestination(flight_row: Flight): Boolean = {
    return flight_row.ORIGIN_COUNTRY_NAME == flight_row.DEST_COUNTRY_NAME
}

originIsDestination: (flight_row: Flight)Boolean


In [18]:
flights.collect().filter(flight_row => originIsDestination(flight_row))

res10: Array[Flight] = Array(Flight(United States,United States,348113))


## 11.4.2 매핑

필터링은 단순한 트랜스포메이션임<br/>
때로는 특정 값을 다른 값으로 매핑해야 함<br/>
이전 예제에서 함수를 정의하면서 이미 매핑 작업을 해봤음<br/>
정의한 함수는 Flight 데이터 타입을 입력으로 사용해 불리언값을 반환함<br/>
하지만 실무에서는 값을 추출하거나 값을 비교하는 것과 같은 정교한 처리가 필요함<br/>

Dataset을 다루는 가장 간단한 예제는 로우의 특정 컬럼값을 추출하는 것임<br/>
DataFrame에 매핑 작업을 수행하는 것은 Datset의 select 메서드를 사용하는 것과 같음<br/>
다음은 목적지 컬럼을 추출하는 예제임<br/>

In [19]:
val destinations = flights.map(f => f.DEST_COUNTRY_NAME)

destinations: org.apache.spark.sql.Dataset[String] = [value: string]


최종적으로 String 데이터 타입의 Dataset을 반환함<br/>
스파크는 결과로 반환할 JVM 데이터 타입을 알고 있기 때문에 컴파일 타임에 데이터 타입의 유효성을 검사할 수 있음<br/>

드라이버는 결과값을 모아 문자열 타입의 배열로 반환함<br/>

In [20]:
val localDestinations = destinations.take(5)

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8) (192.168.0.7 executor driver): java.lang.ClassCastException: $iw cannot be cast to $iw

*위 에러는 왜 생기는 것인지 모르겠음*<br/>

# 11.5 조인

조인은 DataFrame에서와 마찬가지로 Dataset에도 동일하게 적용됨<br/>
하지만 Dataset은 joinWith처럼 정교한 메서드를 제공함<br/>
joinWith 메서드는 co-group(RDD 용어)과 거의 유사하며 Dataset 안쪽에 다른 두 개의 중첩된 Dataset으로 구성됨<br/>
각 컬럼은 단일 Dataset이므로 Dataset 객체를 컬럼처럼 다룰 수 있음<br/>
그러므로 조인 수행 시 더 많은 정보를 유지할 수 있으며 고급 맵이나 필터처럼 정교하게 데이터를 다룰 수 있음<br/>

joinWith 메서드를 설명하기 위해 가짜 항공운항 메타데이터 Dataset을 생성함<br/>

In [42]:
import spark.implicits._ // 이 라인을 추가해야 실행됨

case class FlightMetadata(count: BigInt, randomData: BigInt)

val flightsMeta = spark.range(500).map(x => (x, scala.util.Random.nextLong))
  .withColumnRenamed("_1", "count").withColumnRenamed("_2", "randomData")
  .as[FlightMetadata]

val flights2 = flights
  .joinWith(flightsMeta, flights.col("count") === flightsMeta.col("count"))

import spark.implicits._
defined class FlightMetadata
flightsMeta: org.apache.spark.sql.Dataset[FlightMetadata] = [count: bigint, randomData: bigint]
flights2: org.apache.spark.sql.Dataset[(Flight, FlightMetadata)] = [_1: struct<DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field>, _2: struct<count: bigint, randomData: bigint>]


최종적으로 로우는 Flight와 FlightMetadata로 이루어진 일종의 키-값 형태의 Dataset을 반환함<br/>
Dataset이나 복합 데이터 타입의 DataFrame으로 데이터를 조회할 수 있음<br/>

In [44]:
flights2.show(false)

+------------------------------------+------------------------+
|_1                                  |_2                      |
+------------------------------------+------------------------+
|{United States, Uganda, 1}          |{1, 2795269918196455456}|
|{United States, French Guiana, 1}   |{1, 2795269918196455456}|
|{Bulgaria, United States, 1}        |{1, 2795269918196455456}|
|{United States, Slovakia, 1}        |{1, 2795269918196455456}|
|{United States, Cameroon, 1}        |{1, 2795269918196455456}|
|{United States, Burkina Faso, 1}    |{1, 2795269918196455456}|
|{United States, Indonesia, 1}       |{1, 2795269918196455456}|
|{Vietnam, United States, 1}         |{1, 2795269918196455456}|
|{United States, Papua New Guinea, 1}|{1, 2795269918196455456}|
|{United States, Bahrain, 1}         |{1, 2795269918196455456}|
|{United States, Gabon, 1}           |{1, 2795269918196455456}|
|{United States, Liberia, 1}         |{1, 2795269918196455456}|
|{Malaysia, United States, 1}        |{1

In [45]:
flights2.selectExpr("_1.DEST_COUNTRY_NAME").show()

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|         Bulgaria|
|    United States|
|    United States|
|    United States|
|    United States|
|          Vietnam|
|    United States|
|    United States|
|    United States|
|    United States|
|         Malaysia|
|    United States|
|    United States|
|    United States|
|    United States|
|          Liberia|
|       Azerbaijan|
|    United States|
+-----------------+
only showing top 20 rows



드라이버로 데이터를 모은 다음 결과를 반환함<br/>

In [46]:
flights2.take(2)

res23: Array[(Flight, FlightMetadata)] = Array((Flight(United States,Uganda,1),FlightMetadata(1,-5337248526406846490)), (Flight(United States,French Guiana,1),FlightMetadata(1,-5337248526406846490)))


일반 조인 역시 잘 동작함<br/>
하지만 DataFrame을 반환하므로 JVM 데이터 타입 정보를 모두 잃게 됨<br/>

In [47]:
val flights2 = flights.join(flightsMeta, Seq("count"))

flights2: org.apache.spark.sql.DataFrame = [count: bigint, DEST_COUNTRY_NAME: string ... 2 more fields]


이 정보를 다시 얻으려면 다른 Dataset을 정의해야 함<br/>
DataFrame과 Dataset을 조인하는 것은 아무런 문제가 되지 않으며 최종적으로 동일한 결과를 반환함<br/>

In [48]:
val flights2 = flights.join(flightsMeta.toDF(), Seq("count"))

flights2: org.apache.spark.sql.DataFrame = [count: bigint, DEST_COUNTRY_NAME: string ... 2 more fields]


# 11.6 그룹화와 집계

그룹화와 집계는 이전 장에서 보았던 것과 동일한 표준을 따르므로 groupBy, rollup 그리고 cube 메서드를 여전히 사용할 수 있음<br/>
하지만 Dataset 대신 DataFrame을 반환하기 때문에 데이터 타입 정보를 잃게 됨<br/>

In [49]:
flights.groupBy("DEST_COUNTRY_NAME").count()

res24: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, count: bigint]


데이터 타입 정보를 잃는 것은 그다지 큰 문제는 아니지만, 이를 유지할 수 있는 그룹화와 집계 방법이 있음<br/>
한 가지 예로 groupByKey 메서드는 Dataset의 특정 키를 기준으로 그룹화하고 형식화된 Dataset을 반환함<br/>
하지만 이 함수는 컬럼명 대신 함수를 파라미터로 사용해야 함<br/>
따라서 다음 예제와 같이 훨씬 더 정교한 그룹화 함수를 사용할 수 있음<br/>

In [51]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).count()

res26: org.apache.spark.sql.Dataset[(String, Long)] = [key: string, count(1): bigint]


groupByKey 메서드의 파라미터로 함수를 사용함으로써 유연성을 얻을 수 있음<br/>
하지만 스파크는 함수와 JVM 데이터 타입을 최적화할 수 없으므로 trade-off가 발생함<br/>
이로 인해 성능 차이가 발생할 수 있으며 실행 계획으로 그 이유를 확인할 수 있음<br/>
다음 예제는 groupByKey 메서드를 사용해 DataFrame에 새로운 컬럼을 추가한 다음 그룹화를 수행함<br/>

In [52]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).count().explain

== Physical Plan ==
*(3) HashAggregate(keys=[value#309], functions=[count(1)])
+- Exchange hashpartitioning(value#309, 200), ENSURE_REQUIREMENTS, [id=#608]
   +- *(2) HashAggregate(keys=[value#309], functions=[partial_count(1)])
      +- *(2) Project [value#309]
         +- AppendColumns $line82.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$Lambda$4094/798789902@1be80bc4, newInstance(class $line8.$read$$iw$$iw$Flight), [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false) AS value#309]
            +- *(1) ColumnarToRow
               +- FileScan parquet [DEST_COUNTRY_NAME#0,ORIGIN_COUNTRY_NAME#1,count#2L] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/home/hj/Downloads/Spark-The-Definitive-Guide/data/flight-data/parquet/201..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,co

Dataset의 키를 이용해 그룹화를 수행한 다음 결과를 키-값 형태로 함수에 전달해 원시 객체 형태로 그룹화된 데이터를 다룰 수 있음<br/>

In [54]:
def grpSum(countryName:String, values: Iterator[Flight]) = {
  values.dropWhile(_.count < 5).map(x => (countryName, x))
}
flights.groupByKey(x => x.DEST_COUNTRY_NAME).flatMapGroups(grpSum).show(5)

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 25.0 failed 1 times, most recent failure: Lost task 0.0 in stage 25.0 (TID 24) (192.168.0.7 executor driver): java.lang.ClassCastException: $iw cannot be cast to $iw

*위와 같은 에러인데 마찬가지로 왜 생기는지 모르겠음*

In [57]:
flights.groupBy("DEST_COUNTRY_NAME").count().explain

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#0], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#0, 200), ENSURE_REQUIREMENTS, [id=#816]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#0], functions=[partial_count(1)])
      +- *(1) ColumnarToRow
         +- FileScan parquet [DEST_COUNTRY_NAME#0] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/home/hj/Downloads/Spark-The-Definitive-Guide/data/flight-data/parquet/201..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




groupByKey 메서드는 동일한 결과를 반환하지만 데이터 스캔 직후에 집계를 수행하는 groupBy 메서드에 비해 더 비싼 처리라는 것을 실행 계획으로 알 수 있음<br/>

# 11.7 정리

이 장에서는 Dataset의 기초와 Dataset 사용이 적합한 경우를 예제와 함께 알아보았음<br/>
다시 말해, Dataset을 사용하기 위해 기본적으로 알아야 하는 모든 내용과 Dataset 사용 방법에 대해 알아보았음<br/>
Dataset은 고수준의 구조적 API와 다음 장의 주제인 저수준 RDD API가 조합된 형태라고 생각하면 쉽게 이해할 수 있음<br/>